# Deep Learning Toolkit for Splunk - Drift Detection

This notebook contains an example workflow how to work on custom containerized code that seamlessly interfaces with the Deep Learning Toolkit for Splunk.

Note: By default every time you save this notebook the cells are exported into a python module which is then invoked by Splunk MLTK commands like <code> | fit ... | apply ... | summary </code>. Please read the Model Development Guide in the Deep Learning Toolkit app for more information.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [1]:
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import numpy as np
import pandas as pd
import datetime as dt
import bocd
# ...
# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

In [3]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)

numpy version: 1.19.2
pandas version: 1.1.3


## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a dataset into your notebook environment. Note: mode=stage is used in the | fit command to do this.

| inputlookup logins.csv <br>
| fit MLTKContainer mode=stage algo=drift_detection m=96 logins from _time into app:drift_detection<br>

After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [4]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [5]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
df, param = stage("drift_detection")
print(df.describe())
print(param)

              _time      logins
count  7.080000e+02  708.000000
mean   1.447645e+09  146.108757
std    7.362946e+05  115.425482
min    1.446372e+09    5.000000
25%    1.447008e+09   60.000000
50%    1.447645e+09   98.000000
75%    1.448281e+09  220.250000
max    1.448917e+09  540.000000
{'options': {'params': {'mode': 'stage', 'algo': 'drift_detection', 'm': '96'}, 'args': ['logins', '_time'], 'target_variable': ['logins'], 'feature_variables': ['_time'], 'model_name': 'drift_detection', 'algo_name': 'MLTKContainer', 'mlspl_limits': {'disabled': False, 'handle_new_cat': 'default', 'max_distinct_cat_values': '10000', 'max_distinct_cat_values_for_classifiers': '10000', 'max_distinct_cat_values_for_scoring': '10000', 'max_fit_time': '6000', 'max_inputs': '100000000', 'max_memory_usage_mb': '4000', 'max_model_size_mb': '150', 'max_score_time': '6000', 'streaming_apply': '0', 'use_sampling': '1'}, 'kfold_cv': None}, 'feature_variables': ['_time'], 'target_variables': ['logins']}


## Stage 2 - create and initialize a model

In [6]:
# initialize your model
# available inputs: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    model = {}
    return model

In [7]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
model = init(df,param)
print(model)

{}


## Stage 3 - fit the model

In [8]:
# train your model
# returns a fit info json object and may modify the model object
def fit(model,df,param):
    # model.fit()
    info = {"message": "model created"}
    return info

In [9]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(fit(model,df,param))

{'message': 'model created'}


## Stage 4 - apply the model

In [62]:
# apply your model
# returns the calculated results
def apply(model,df,param):
    p_hazard = 200
    p_mu=0
    p_kappa=1
    p_alpha=1
    p_beta=1
    if 'options' in param:
        if 'params' in param['options']:
            if 'hazard' in param['options']['params']:
                p_hazard = int(param['options']['params']['hazard'])
            if 'mu' in param['options']['params']:
                p_mu = float(param['options']['params']['mu'])
            if 'kappa' in param['options']['params']:
                p_kappa = float(param['options']['params']['kappa'])
            if 'alpha' in param['options']['params']:
                p_alpha = float(param['options']['params']['alpha'])
            if 'beta' in param['options']['params']:
                p_beta = float(param['options']['params']['beta'])
    target = param['target_variables'][0]
    signal = df[target].to_numpy()
    
    # Initialize object
    detector = bocd.BayesianOnlineChangePointDetection(bocd.ConstantHazard(p_hazard), bocd.StudentT(mu=p_mu, kappa=p_kappa, alpha=p_alpha, beta=p_beta))

    # Online estimation and get the maximum likelihood r_t at each time point
    rt_mle = np.empty(signal.shape)
    for i, d in enumerate(signal):
        detector.update(d)
        rt_mle[i] = detector.rt
    
    df['drift'] = 0
    index_changes = np.where(np.diff(rt_mle)<0)[0]
    for i in index_changes:
        df.at[i, 'drift'] = 1
   
    return df

In [63]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(apply(model,df,param))

          _time  logins  drift
0    1446372000      39      0
1    1446375600      27      0
2    1446379200      49      0
3    1446382800      59      0
4    1446386400      62      0
..          ...     ...    ...
703  1448902800     540      0
704  1448906400     527      0
705  1448910000     433      0
706  1448913600     411      0
707  1448917200     475      0

[708 rows x 3 columns]


## Stage 5 - save the model

In [33]:
# save model to name in expected convention "<algo_name>_<model_name>"
def save(model,name):
    with open(MODEL_DIRECTORY + name + ".json", 'w') as file:
        json.dump(model, file)
    return model

## Stage 6 - load the model

In [34]:
# load model from name in expected convention "<algo_name>_<model_name>"
def load(name):
    model = {}
    with open(MODEL_DIRECTORY + name + ".json", 'r') as file:
        model = json.load(file)
    return model

## Stage 7 - provide a summary of the model

In [35]:
# return a model summary
def summary(model=None):
    returns = {"version": {"numpy": np.__version__, "pandas": pd.__version__} }
    return returns

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code